In [ ]:
# ========================== Imports 

import pandas as pd
import numpy as np
import GetPoolData
import itertools
import scipy
from scipy import stats
# Create config.py in this directory and enter your own Bitquery API Token
from config import BITQUERY_API_TOKEN

import ResetStrategy
import ActiveStrategyFramework


In [ ]:


DOWNLOAD_DATA      = False
address            = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'
token_0_address    = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
token_1_address    = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
flipside_queries   = ['https://api.flipsidecrypto.com/api/v2/queries/b8ad3087-803a-478b-9ed3-c4f3c096bc47/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/de277680-5ff6-4d58-bfff-29ef114215be/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/669fc9f7-8b2d-4dd2-948f-668de9eb4e8a/data/latest']


price_data_begin,price_data_end  = "2022-01-01","2022-01-07"
file_name                = 'eth_usdc'

swap_data                = GetPoolData.get_pool_data_flipside(address,flipside_queries,file_name,DOWNLOAD_DATA)
price_data               = GetPoolData.get_price_data_bitquery(token_0_address,token_1_address,price_data_begin,price_data_end,BITQUERY_API_TOKEN,file_name,DOWNLOAD_DATA)
# If price data not being downloaded, initialize the flag seperately.
DATE_BEGIN                 = pd.to_datetime(f'{price_data_begin} 00:00PM',utc=True)
DATE_END                   = pd.to_datetime(f'{price_data_end} 00:00PM',utc=True)


# Data for strategy simulation. We can use aggregate_price_data to analyze the strategy at a coarser STRATEGY_FREQUENCY in minutes
z_score_cutoff                           = 5
window_size                              = 60*12
STRATEGY_FREQUENCY                       = '10 min' 
simulate_data_filtered                   = ActiveStrategyFramework.aggregate_price_data(price_data,STRATEGY_FREQUENCY)
simulate_data_filtered_roll              = simulate_data_filtered.quotePrice.rolling(window=window_size) 
simulate_data_filtered['roll_median']    = simulate_data_filtered_roll.median()
roll_dev                                 = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median)
simulate_data_filtered['median_abs_dev'] = 1.4826*roll_dev.rolling(window=window_size).median() # sd(y)/mad(y, const=1) = 1.4826
outlier_indices                          = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median) >= z_score_cutoff*simulate_data_filtered['median_abs_dev']
simulate_data_price                      = simulate_data_filtered[~outlier_indices]['quotePrice'][DATE_BEGIN:DATE_END]

# Data for strategy estimation
STATISTICAL_FREQUENCY        = '8H' 
statistical_data             = ActiveStrategyFramework.aggregate_price_data(price_data,STATISTICAL_FREQUENCY)
statistical_data['z_scores'] = np.abs(scipy.stats.zscore(statistical_data['quotePrice']))
statistical_data             = statistical_data.drop(statistical_data[abs(statistical_data.z_scores) > 3].index)
statistical_data_filtered    = statistical_data.sort_index()

# Initial Position Details
INITIAL_TOKEN_0            = 100000
INITIAL_TOKEN_1            = INITIAL_TOKEN_0*simulate_data_price[0]
FEE_TIER                   = 0.003

# Set decimals according to your pool
DECIMALS_0 = 6
DECIMALS_1 = 18
swap_data['virtual_liquidity'] = swap_data['VIRTUAL_LIQUIDITY_ADJUSTED']*(10**((DECIMALS_1  + DECIMALS_0)/2))
swap_data['traded_in']         = swap_data.apply(lambda x: -x['amount0'] if (x['amount0'] < 0) else -x['amount1'],axis=1).astype(float)
swap_data['traded_out']        = swap_data.apply(lambda x:  x['amount0'] if (x['amount0'] > 0) else  x['amount1'],axis=1).astype(float)



sim_performance  = []
sim_results      = []

x, y = 0.75,0.5 

Reset_Strategy                   = ResetStrategy.ResetStrategy(statistical_data_filtered,x,y)
simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,swap_data,Reset_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
print(simulated_strategy[-1].liquidity_in_1,simulated_strategy[-1].liquidity_in_0,"-",INITIAL_TOKEN_0,INITIAL_TOKEN_1)
sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,Reset_Strategy)
strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data)
sim_data.columns
strat_result['alpha_param']      = x
strat_result['tau_param']        = y
# strat_result['limit_param']      =
sim_results.append(simulated_strategy)
sim_performance.append(strat_result)

In [ ]:

print(
    simulated_strategy[-1].liquidity_in_0+simulated_strategy[-1].token_0_left_over+simulated_strategy[-1].token_0_fees_uncollected,
    simulated_strategy[-1].liquidity_in_1+simulated_strategy[-1].token_1_left_over+simulated_strategy[-1].token_1_fees_uncollected
    )

In [ ]:

#################
#
# Download data
#
################

# Get Swap Data
DOWNLOAD_DATA      = False
address            = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'
token_0_address    = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
token_1_address    = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
flipside_queries   = ['https://api.flipsidecrypto.com/api/v2/queries/b8ad3087-803a-478b-9ed3-c4f3c096bc47/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/de277680-5ff6-4d58-bfff-29ef114215be/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/669fc9f7-8b2d-4dd2-948f-668de9eb4e8a/data/latest']

# Price data for theThese must be month ends
# ["5-11","12-18","19-25","26-1","2-8","9-15","16-22","23-29","30-6","7-13","16-20","21-27","28-3","4-10","11-17","18-24","25-1"]

dates = [
    ["2022-01-01","2022-01-07"],
    ["2022-01-08","2022-01-14"],
    ["2022-01-15","2022-01-21"],
    ["2022-01-22","2022-01-28"],
    ["2022-01-29",
    
    "2022-02-04"],
    ["2022-02-05","2022-02-11"],
    ["2022-02-12","2022-02-18"],
    ["2022-02-19","2022-02-25"],
    ["2022-02-26",
    
    "2022-03-04"],
    ["2022-03-05","2022-03-11"],
    ["2022-03-12","2022-03-18"],
    ["2022-03-19","2022-03-25"],
    ["2022-03-26",
    
    "2022-04-01"],
    ["2022-04-02","2022-04-08"],
    ["2022-04-09","2022-04-15"],
    ["2022-04-16","2022-04-22"],
    ["2022-04-23","2022-04-29"],
    ["2022-04-30",
    
    "2022-05-06"],
    ["2022-05-07","2022-05-13"],
    ["2022-05-16","2022-05-20"],
    ["2022-05-21","2022-05-27"],
    ["2022-05-28",
    
    "2022-06-03"],
    ["2022-06-04","2022-06-10"],
    ["2022-06-11","2022-06-17"],
    ["2022-06-18","2022-06-24"],
    ["2022-06-25",
    
    "2022-07-1"],
]


for date in dates:
    start = date[0]
    end = date[1]
    print(date)

    price_data_begin         = start
    price_data_end           = end
    file_name                = 'eth_usdc'

    swap_data                = GetPoolData.get_pool_data_flipside(address,flipside_queries,file_name,DOWNLOAD_DATA)
    price_data               = GetPoolData.get_price_data_bitquery(token_0_address,token_1_address,price_data_begin,price_data_end,BITQUERY_API_TOKEN,file_name,DOWNLOAD_DATA)
    # If price data not being downloaded, initialize the flag seperately.


    DATE_BEGIN                 = pd.to_datetime(f'{price_data_begin} 00:00PM',utc=True)
    DATE_END                   = pd.to_datetime(f'{price_data_end} 00:00PM',utc=True)


    # Data for strategy simulation. We can use aggregate_price_data to analyze the strategy at a coarser STRATEGY_FREQUENCY in minutes
    z_score_cutoff                           = 5
    window_size                              = 60*12
    STRATEGY_FREQUENCY                       = '10 min' 
    simulate_data_filtered                   = ActiveStrategyFramework.aggregate_price_data(price_data,STRATEGY_FREQUENCY)
    simulate_data_filtered_roll              = simulate_data_filtered.quotePrice.rolling(window=window_size) 
    simulate_data_filtered['roll_median']    = simulate_data_filtered_roll.median()
    roll_dev                                 = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median)
    simulate_data_filtered['median_abs_dev'] = 1.4826*roll_dev.rolling(window=window_size).median() # sd(y)/mad(y, const=1) = 1.4826
    outlier_indices                          = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median) >= z_score_cutoff*simulate_data_filtered['median_abs_dev']
    simulate_data_price                      = simulate_data_filtered[~outlier_indices]['quotePrice'][DATE_BEGIN:DATE_END]

    # Data for strategy estimation
    STATISTICAL_FREQUENCY        = '8H' 
    statistical_data             = ActiveStrategyFramework.aggregate_price_data(price_data,STATISTICAL_FREQUENCY)
    statistical_data['z_scores'] = np.abs(scipy.stats.zscore(statistical_data['quotePrice']))
    statistical_data             = statistical_data.drop(statistical_data[abs(statistical_data.z_scores) > 3].index)
    statistical_data_filtered    = statistical_data.sort_index()

    # Initial Position Details
    INITIAL_TOKEN_0            = 100000
    INITIAL_TOKEN_1            = INITIAL_TOKEN_0*simulate_data_price[0]
    FEE_TIER                   = 0.003

    # Set decimals according to your pool
    DECIMALS_0 = 6
    DECIMALS_1 = 18
    swap_data['virtual_liquidity'] = swap_data['VIRTUAL_LIQUIDITY_ADJUSTED']*(10**((DECIMALS_1  + DECIMALS_0)/2))
    swap_data['traded_in']         = swap_data.apply(lambda x: -x['amount0'] if (x['amount0'] < 0) else -x['amount1'],axis=1).astype(float)
    swap_data['traded_out']        = swap_data.apply(lambda x:  x['amount0'] if (x['amount0'] > 0) else  x['amount1'],axis=1).astype(float)



    sim_performance  = []
    sim_results      = []

    x, y = 0.75,0.5 

    Reset_Strategy                   = ResetStrategy.ResetStrategy(statistical_data_filtered,x,y)
    simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,swap_data,Reset_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
    sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,Reset_Strategy)
    strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data)
    sim_data.columns
    strat_result['alpha_param']      = x
    strat_result['tau_param']        = y
    # strat_result['limit_param']      =
    sim_results.append(simulated_strategy)
    sim_performance.append(strat_result)

    pd.DataFrame(sim_performance).to_csv(f"./{price_data_begin}--{price_data_end}-{STRATEGY_FREQUENCY}-{STATISTICAL_FREQUENCY}.csv")

    with open(f"./{price_data_begin}--{price_data_end}-{STRATEGY_FREQUENCY}-{STATISTICAL_FREQUENCY}.txt","w") as file:
        file.write(pd.DataFrame(sim_performance).to_string())


In [ ]:
price_data

In [ ]:
#################
#
# Run Reset Strategy
#
################

# Select date ranges for strategy simulation
# DATE_BEGIN                 = pd.to_datetime(f'2022-06-01 00:00PM',utc=True)
# DATE_END                   = pd.to_datetime(f'2022-06-20 00:00PM',utc=True)

# Strategy Parameters
# alpha_range = [.50]
# tau_range   = [.50]
# limit_range = [.40]



In [ ]:
pd.DataFrame(simulate_data_filtered)


In [ ]:

sim_performance  = []
sim_results      = []
# for j in list(itertools.product(alpha_range,tau_range)):
# for x in np.linspace(0.1,1,num=19):
# for x in np.linspace(0.5,0.6,num=3):
for x in np.linspace(0.5,0.95,num=10):
    # for y in [round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(x).__round__(2) ]:
    for y in [round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(x).__round__(2) ]:
        print(x,y)
        Reset_Strategy                   = ResetStrategy.ResetStrategy(statistical_data_filtered,x,y)
        simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,swap_data,Reset_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
        sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,Reset_Strategy)
        strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data)
        sim_data.columns
        strat_result['alpha_param']      = x
        strat_result['tau_param']        = y
        # strat_result['limit_param']      =
        sim_results.append(simulated_strategy)
        sim_performance.append(strat_result)
    print("")
    

In [ ]:
df = pd.DataFrame(sim_performance)
df

In [ ]:
df["net_return"].max()

In [ ]:
df.iloc[df["net_return"].argmax(),:]

In [ ]:
reset = ResetStrategy.ResetStrategy(statistical_data_filtered,0.5,0.4)

base_range_lower      = (1 + reset.inverse_ecdf((1 -      reset.alpha_param)/2))  * simulate_data_price[0]
base_range_upper      = (1 + reset.inverse_ecdf( 1 - (1 - reset.alpha_param)/2))  * simulate_data_price[0]

print(base_range_lower, base_range_upper)

In [ ]:
import math

decimal_adjustment          = 10**(18  - 6)
tickSpacing = 60

# Lower Range
TICK_A_PRE         = int(math.log(decimal_adjustment*base_range_lower,1.0001))
TICK_A             = int(round(TICK_A_PRE/tickSpacing)*tickSpacing)

# Upper Range
TICK_B_PRE        = int(math.log(decimal_adjustment*base_range_upper,1.0001))
TICK_B            = int(round(TICK_B_PRE/tickSpacing)*tickSpacing)


print(TICK_A,TICK_B)

In [ ]:
eth_A = 1 / ((1.0001**TICK_A)/10**18 * 10**6)
eth_B = 1 / ((1.0001**TICK_B)/10**18 * 10**6)

print(eth_A,eth_B)

In [ ]:
import seaborn as sns
# doosra
sns.set_style("darkgrid", {'axes.labelcolor': 'white'})
sns.ecdfplot(x = 'price_return', data = statistical_data_filtered)

In [ ]:
for j in list(itertools.product([1,11,111],[2])):
    print(j[0],j[1])

In [ ]:
for i in np.linspace(0.5,0.95,num=10):
    print(float(i).__round__(2))

In [ ]:
[round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(0.5).__round__(2) ]

In [ ]:
import pickle
import json



In [ ]:
with open('./data/eth_usdc_swap.pkl', 'rb') as input:
    request_swap = pickle.load(input)

request_swap


with open('./swap.json', 'w') as file:
    file.writelines(json.dumps(request_swap))

In [ ]:
json.dumps(request_swap)

In [20]:
token0 = 4386
token1 = 1

tick = 204578


In [21]:
token0Price = ((1.0001**tick)/10**18 * 10**6)
token1Price = 1/token0Price

print(token0Price,token1Price)


0.000766064691799601 1305.3727847067978


In [22]:


swapFee = 0.003*token0

swapableAmounts = token0-swapFee

swapedAmount = swapableAmounts*token0Price

swapedAmount


3.3498798590183507

In [14]:
def swapAmounts(cTick,oAmount,direction,percSwap = 1):
    token0Price = ((1.0001**cTick)/10**18 * 10**6)
    token1Price = 1/token0Price

    amount = oAmount *percSwap

    swapFee = 0.003*amount
    swapableAmounts = amount-swapFee

    return ((swapableAmounts*token0Price,amount,swapFee) \
        if direction < 0 else \
            (swapableAmounts*token1Price,amount,swapFee))


In [10]:
swapAmounts(tick,4386,-1)

(3.3148918782032384, 13.158)

In [11]:
# 10 12 20

lower = 204178
upper = 204778


width = upper - lower
current = tick - (lower)

ratio = current/(width)

print(ratio,width,current)

usdcs = (token1Price*token1)+token0

print(usdcs*ratio,swapAmounts(tick,usdcs-(usdcs*ratio),-1))




0.49166666666666664 600 295


NameError: name 'token1' is not defined

In [4]:
import UNI_v3_funcs


In [5]:
# L = 15324049559134797569
INITIAL_TOKEN_1            = 3
INITIAL_TOKEN_0            = 999*INITIAL_TOKEN_1


d0 = 6
d1 = 18


tick = 204473

lower = 201473
upper = 207473

token0Price = ((1.0001**tick)/10**18 * 10**6)
token1Price = 1/token0Price

print(token0Price,token1Price)

0.0007580634923931024 1319.1507176307057


In [6]:
L = int(UNI_v3_funcs.get_liquidity(tick,lower,upper,INITIAL_TOKEN_0,INITIAL_TOKEN_1,d0,d1))

In [16]:
import numpy as np

amt0, amt1 = UNI_v3_funcs.get_amounts(tick,lower,upper,L,d0,d1)

print(amt0, amt1)

rec = []

for i in np.linspace(0.05,1,num=20):

    sAmount,amt,_ = swapAmounts(tick,INITIAL_TOKEN_1-amt1,1,i)
    LL = int(UNI_v3_funcs.get_liquidity(tick,lower,upper,INITIAL_TOKEN_0+sAmount,INITIAL_TOKEN_1-amt,d0,d1))
    rec.append(
        UNI_v3_funcs.get_amounts(tick,lower,upper,LL,d0,d1)
    )

print(rec)

2996.999999999998 2.271916286702129
[(3044.878539821669, 2.3082112598100277), (3092.75707964334, 2.3445062329179254), (3140.6356194650157, 2.3808012060258275), (3188.514159286686, 2.417096179133725), (3236.3926991083576, 2.453391152241623), (3284.271238930028, 2.489686125349521), (3332.1497787517037, 2.525981098457423), (3380.0283185733742, 2.5622760715653206), (3427.9068583950448, 2.598571044673219), (3475.7853982167157, 2.6348660177811167), (3429.203468801447, 2.5995539576861693), (3381.1808611568435, 2.5631497720212777), (3333.158253512236, 2.526745586356382), (3285.135645867633, 2.49034140069149), (3237.1130382230244, 2.4539372150265955), (3189.090430578421, 2.4175330293617034), (3141.067822933813, 2.381128843696808), (3093.04521528921, 2.3447246580319168), (3045.0226076446015, 2.308320472367021), (2996.999999999998, 2.271916286702129)]


In [1]:

    # print(float(i).__round__(2))

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
1.0
